# Calculate Limits

## Simulate First

You can just run the whole notebook

### Imports / Helper Functions

In [1]:
import sys
import math
import warnings

import psycopg2
import wrds
import gzip

import seaborn as sns
import os
import quandl
import json
import zipfile
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import functools
import requests
import io

import urllib.request
from urllib.error import HTTPError
# from html_table_parser.parser import HTMLTableParser
import re

import plotnine as p9
from plotnine import ggplot, scale_x_date, guides, guide_legend, geom_bar, scale_y_continuous, \
    scale_color_identity, geom_line, geom_point, labs, theme_minimal, theme, element_blank, element_text, \
        geom_ribbon, geom_hline, aes, scale_size_manual, scale_color_manual, ggtitle

from datetime import datetime
import datetime

import pandas as pd
# import pandas_market_calendars as mcal
from pandas.plotting import autocorrelation_plot
import numpy as np
from numpy import cumsum, log, polyfit, sqrt, std, subtract
import scipy as sp
from scipy.stats import norm
import scipy.stats as stats

from statsmodels.tsa.stattools import coint
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
from statsmodels.tsa.stattools import acf
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant

from collections import deque
from bisect import insort, bisect_left
from itertools import islice

pd.set_option('display.max_columns', None)

In [2]:
def max_drawdown(returns):
        local_max = [n for n in range(len(returns)-1) if ((n==0) and (returns[0] > returns[1])) or 
           ((n > 0) and  (returns[n-1]<returns[n]) and (returns[n+1]<returns[n]))] 
        
        local_min = [n for n in range(1,len(returns)) if ((n == len(returns)-1) and (returns[-1] < returns[-2])) or
                (returns[n-1]>returns[n]) and (returns[n+1]>returns[n])]
        
        def next_local_min(n):
            if [m for m in local_min if m > n]:
                return [m for m in local_min if m > n][0]
            else: return None
        
        drawdowns = [(n,next_local_min(n)) for n in local_max]
        drawdown_values = [returns[n] - returns[m] for (n,m) in drawdowns if m != None]
        if drawdown_values:
            return  np.max(drawdown_values)
        else: return 0.0

### Dividends

In [3]:
start_date = '2018-01-01'
end_date = '2023-02-28'
spy_divdata = pd.read_csv('spy_tickerdata.csv')[['date','dividend']].sort_values(by='date').reset_index(drop=True)
spy_divdata = spy_divdata.loc[(spy_divdata['date'] >= start_date) & (spy_divdata['date'] <= end_date)].copy().reset_index(drop=True)
spy_divdata['date'] = pd.to_datetime(spy_divdata['date'])
trading_days = spy_divdata['date']
spy_divdata = spy_divdata.loc[spy_divdata['dividend'] != 0]

# Step 1: Calculate the last day of the following month
spy_divdata['date'] = pd.to_datetime(spy_divdata['date'])  # Ensure 'date' is in datetime format
spy_divdata['end_of_next_month'] = spy_divdata['date'] + pd.offsets.MonthEnd(2)

# Step 2: Adjust to the nearest trading day

# This function finds the last trading day of the month or the nearest previous trading day
def find_pay_date(end_of_month, trading_days):
    if end_of_month in trading_days:
        return end_of_month
    else:
        # Find the nearest previous trading day
        eligible_days = trading_days[trading_days <= end_of_month]
        return eligible_days.max()  # The last trading day before or on the end_of_month

# Apply the function to each row to determine the 'pay_date'
spy_divdata['pay_date'] = spy_divdata['end_of_next_month'].apply(lambda date: find_pay_date(date, trading_days))

# Optionally, you can drop the intermediate 'end_of_next_month' column if it's no longer needed
spy_divdata.drop(columns=['end_of_next_month'], inplace=True)

spy_divdata

,date,dividend,pay_date
51,2018-03-16,1.096780,2018-04-30
114,2018-06-15,1.246000,2018-07-31
182,2018-09-21,1.323000,2018-10-31
245,2018-12-21,1.435400,2019-01-31
301,2019-03-15,1.233100,2019-04-30
369,2019-06-21,1.431600,2019-07-31
432,2019-09-20,1.383620,2019-10-31
496,2019-12-20,1.570000,2020-01-31
557,2020-03-20,1.405560,2020-04-30
620,2020-06-19,1.366200,2020-07-31


### Simulations

In [4]:
data = pd.read_csv('combinedata.csv')
options = pd.read_csv('option_df.csv')

data['exdate'] = pd.to_datetime(data['exdate'])
options['exdate'] = pd.to_datetime(options['exdate'])


data['exdate_str'] = data['exdate'].dt.strftime('%Y%m%d')  
data['strikeID'] = data['exdate_str'] + '_' + data['strike_price'].astype(str)  
data.drop(columns=['exdate_str'], inplace=True)  

options['exdate_str'] = options['exdate'].dt.strftime('%Y%m%d')  
options['strikeID'] = options['exdate_str'] + '_' + options['strike_price'].astype(str)  
options.drop(columns=['exdate_str'], inplace=True)  

options['date'] = pd.to_datetime(options['date'])
data['date'] = pd.to_datetime(data['date'])

In [5]:
def create_simulations(options_subset, data, dropna_greeks=False):
    simulations = {}

    for index, row in options_subset.iterrows():
        strikeID = row['exdate'].strftime('%Y%m%d') + '_' + str(row['strike_price'])
        mask = (data['strikeID'] == strikeID) & (data['date'] >= row['date']) & (data['date'] <= row['close_date'])
        temp_df = data[mask].sort_values(by=['date', 'cp_flag'])

        shared_cols = ['date', 'exdate', 'strike_price', 'close', 'strikeID'] # 'expiry_indicator',  'adj_open', 'adj_close', 'adj_volume',
        greeks_cols = ['impl_volatility', 'delta'] # , 'gamma', 'vega', 'theta'
        call_specific_cols = ['cp_flag', 'best_bid', 'best_offer'] + greeks_cols # , 'volume', 'open_interest'
        put_specific_cols = call_specific_cols

        calls = temp_df[temp_df['cp_flag'] == 'C'][shared_cols + call_specific_cols].rename(columns={col: col + '_c' for col in call_specific_cols})
        puts = temp_df[temp_df['cp_flag'] == 'P'][shared_cols + put_specific_cols].rename(columns={col: col + '_p' for col in put_specific_cols})

        merged_df = pd.merge(calls, puts, on=shared_cols, how='outer')

        if dropna_greeks:
            greeks_cols_c = [col + '_c' for col in greeks_cols]
            greeks_cols_p = [col + '_p' for col in greeks_cols]
            merged_df = merged_df.dropna(subset=greeks_cols_c + greeks_cols_p, how='any')

        merged_df['delta_sum'] = merged_df['delta_c'].fillna(0) + merged_df['delta_p'].fillna(0)
        merged_df['shares_held'] = -1 * merged_df['delta_sum']

        merged_df = merged_df.sort_values(by='date')
        merged_df['sharechange'] = merged_df['shares_held'].diff()

        simulations[row['date'].strftime('%Y-%m-%d')] = merged_df

    return simulations

In [6]:
%%time
simulations = create_simulations(options, data, dropna_greeks=True)

CPU times: total: 8min 27s
Wall time: 8min 28s


In [7]:
len(trading_days) - len(simulations) # Adds up to contracts with missing greeks

158

This gets rid of simulations missing trading days:

In [8]:
filtered_simulations = {}

for key, df in simulations.items():
    # Ensure 'date' column is in datetime64 dtype
    df['date'] = pd.to_datetime(df['date'])
    
    # Find the range of trading days for each simulation
    start_date = df['date'].min()
    end_date = df['date'].max()
    
    # Generate the expected range of trading days
    expected_trading_days = trading_days[(trading_days >= start_date) & (trading_days <= end_date)]
    
    # Check if all expected trading days are present in the DataFrame
    actual_trading_days = df['date'].unique()
    actual_trading_days = pd.to_datetime(actual_trading_days)  # Ensure this is in datetime format for comparison
    
    # Use .isin() to check if each item in expected_trading_days is in actual_trading_days, then check if all are True
    if expected_trading_days.isin(actual_trading_days).all():
        # If there are no gaps, add the DataFrame to the filtered_simulations dictionary
        filtered_simulations[key] = df

In [9]:
len(trading_days) - len(filtered_simulations)

325

In [10]:
for key, df in list(filtered_simulations.items())[0:1]: 
    print(f"DataFrame for {key}:")
    print(df.columns)
    display(df)  
    print("\n")

DataFrame for 2018-01-03:
Index(['date', 'exdate', 'strike_price', 'close', 'strikeID', 'cp_flag_c',
       'best_bid_c', 'best_offer_c', 'impl_volatility_c', 'delta_c',
       'cp_flag_p', 'best_bid_p', 'best_offer_p', 'impl_volatility_p',
       'delta_p', 'delta_sum', 'shares_held', 'sharechange'],
      dtype='object')


,date,exdate,strike_price,close,strikeID,cp_flag_c,best_bid_c,best_offer_c,impl_volatility_c,delta_c,cp_flag_p,best_bid_p,best_offer_p,impl_volatility_p,delta_p,delta_sum,shares_held,sharechange
0,2018-01-03,2018-02-02,270.0,270.47,20180202_270.0,C,2.65,2.69,0.072398,0.562218,P,1.77,1.80,0.069568,-0.442522,0.119696,-0.119696,NaN
1,2018-01-04,2018-02-02,270.0,271.61,20180202_270.0,C,3.45,3.49,0.077848,0.632690,P,1.43,1.46,0.074706,-0.366922,0.265768,-0.265768,-0.146072
2,2018-01-05,2018-02-02,270.0,273.42,20180202_270.0,C,4.80,4.84,0.083820,0.727658,P,1.02,1.04,0.081783,-0.270119,0.457539,-0.457539,-0.191771
3,2018-01-08,2018-02-02,270.0,273.92,20180202_270.0,C,5.17,5.20,0.089623,0.749240,P,0.81,0.82,0.082307,-0.234696,0.514544,-0.514544,-0.057005
4,2018-01-09,2018-02-02,270.0,274.54,20180202_270.0,C,5.67,5.72,0.093639,0.773171,P,0.83,0.84,0.091492,-0.223283,0.549888,-0.549888,-0.035344
5,2018-01-10,2018-02-02,270.0,274.12,20180202_270.0,C,5.27,5.33,0.092791,0.759070,P,0.85,0.87,0.089866,-0.235707,0.523363,-0.523363,0.026525
6,2018-01-11,2018-02-02,270.0,276.12,20180202_270.0,C,6.76,6.91,0.090857,0.854846,P,0.56,0.58,0.097319,-0.162272,0.692574,-0.692574,-0.169211
7,2018-01-12,2018-02-02,270.0,277.92,20180202_270.0,C,8.45,8.60,0.102281,0.890014,P,0.43,0.44,0.107499,-0.121856,0.768158,-0.768158,-0.075584
8,2018-01-16,2018-02-02,270.0,276.97,20180202_270.0,C,7.82,7.97,0.130638,0.827166,P,0.60,0.61,0.121851,-0.157003,0.670163,-0.670163,0.097995
9,2018-01-17,2018-02-02,270.0,279.61,20180202_270.0,C,10.03,10.25,0.131682,0.904279,P,0.38,0.39,0.135023,-0.101488,0.802791,-0.802791,-0.132628


### PnL and Misc for future calcs

In [11]:
def calculate_realized_PL(df, long_op=True):
    df = df.reset_index(drop=True)
    
    # Vectorized initial operations for stock
    df['stock_pos'] = np.where(long_op, df['shares_held'], -df['shares_held'])
    df = df.drop(columns=['shares_held'])
    df['pos_change'] = np.where(long_op, df['sharechange'], -df['sharechange'])
    df = df.drop(columns=['sharechange'])
    df.loc[0, 'pos_change'] = df.loc[0, 'stock_pos']
    
    df['change_cost_basis'] = df['pos_change'] * df['close']
    df['stock_cost_basis'] = df['change_cost_basis'].cumsum()
    df['daily_stock_value'] = df['stock_pos'] * df['close']
    df['stock_PL'] = df['daily_stock_value'] - df['stock_cost_basis']

    # Initial option value and vectorized daily option value calculation
    df['option_cost_basis'] = df.loc[0, 'best_offer_c'] + df.loc[0, 'best_offer_p'] if long_op else -df.loc[0, 'best_bid_c'] - df.loc[0, 'best_bid_p']
    df['change_cost_basis_op'] = 0.0
    df.loc[0, 'change_cost_basis_op'] = df.loc[0, 'option_cost_basis']
    df['daily_option_value'] = np.where(long_op, df['best_bid_c'] + df['best_bid_p'], -(df['best_offer_c'] + df['best_offer_p']))
    df['option_PL'] = df['daily_option_value']- df['option_cost_basis']

    # Column to track total positions, PL, and cash flow after positions are closed
    df['total_cost_basis'] = df['stock_cost_basis'] + df['option_cost_basis']
    df['total_pos_value'] = df['daily_stock_value'] + df['daily_option_value']
    df['total_PL'] = df['stock_PL'] + df['option_PL']
    df['realized_stock_PL'] = 0.0
    df['realized_option_PL'] = 0.0
    df['realized_PL'] = 0.0

    # Misc
    df['UID'] = df['strikeID'] + '_' + str(df.loc[0, 'date'].date())
    df['to_open'] = 0
    df.loc[0, 'to_open'] = 1
#    df['gross_trades_value'] = abs(df['to_open'] * df['option_cost_basis']) + abs(df['change_cost_basis']) # Need to do this at end

    # Close positions on final day
    final_row_index = len(df) - 1
    df.loc[final_row_index, 'realized_stock_PL'] = df.loc[final_row_index, 'stock_PL']
    df.loc[final_row_index, 'realized_option_PL'] = df.loc[final_row_index, 'option_PL']
    df.loc[final_row_index, 'realized_PL'] = df.loc[final_row_index, 'total_PL']
#    df.loc[final_row_index, 'gross_trades_value'] = abs(df.loc[final_row_index, 'daily_option_value']) + abs(df.loc[final_row_index - 1, 'stock_pos']) * df.loc[final_row_index, 'close'] 

    final_close_price = df.loc[final_row_index, 'close']
    df.loc[final_row_index, 'stock_pos'] = 0
    df.loc[final_row_index, 'pos_change'] = - df.loc[final_row_index - 1, 'pos_change'] if final_row_index > 0 else 0
    df.loc[final_row_index, 'change_cost_basis'] = df.loc[final_row_index, 'pos_change'] * final_close_price
    df.loc[final_row_index, 'stock_cost_basis'] = 0
    df.loc[final_row_index, 'daily_stock_value'] = 0
    df.loc[final_row_index, 'stock_PL'] = 0

    df.loc[final_row_index, 'option_cost_basis'] = 0
    df.loc[final_row_index, 'change_cost_basis_op'] = -df.loc[final_row_index, 'daily_option_value']
    df.loc[final_row_index, 'daily_option_value'] = 0
    df.loc[final_row_index, 'option_PL'] = 0

    df.loc[final_row_index, 'total_cost_basis'] = 0
    df.loc[final_row_index, 'total_pos_value'] = 0
    df.loc[final_row_index, 'total_PL'] = 0
    
    return df

*Cash - total cost basis + net realized PL* gives you how much cash is available to use

In [12]:
simulations_long = {date: calculate_realized_PL(df.copy(), long_op=True) for date, df in filtered_simulations.items()}
simulations_short = {date: calculate_realized_PL(df.copy(), long_op=False) for date, df in filtered_simulations.items()}

In [13]:
for key, df in list(simulations_short.items())[0:1]: 
    print(f"DataFrame for {key}:")
    print(df.columns)
    display(df)  
    print("\n")

DataFrame for 2018-01-03:
Index(['date', 'exdate', 'strike_price', 'close', 'strikeID', 'cp_flag_c',
       'best_bid_c', 'best_offer_c', 'impl_volatility_c', 'delta_c',
       'cp_flag_p', 'best_bid_p', 'best_offer_p', 'impl_volatility_p',
       'delta_p', 'delta_sum', 'stock_pos', 'pos_change', 'change_cost_basis',
       'stock_cost_basis', 'daily_stock_value', 'stock_PL',
       'option_cost_basis', 'change_cost_basis_op', 'daily_option_value',
       'option_PL', 'total_cost_basis', 'total_pos_value', 'total_PL',
       'realized_stock_PL', 'realized_option_PL', 'realized_PL', 'UID',
       'to_open'],
      dtype='object')


,date,exdate,strike_price,close,strikeID,cp_flag_c,best_bid_c,best_offer_c,impl_volatility_c,delta_c,cp_flag_p,best_bid_p,best_offer_p,impl_volatility_p,delta_p,delta_sum,stock_pos,pos_change,change_cost_basis,stock_cost_basis,daily_stock_value,stock_PL,option_cost_basis,change_cost_basis_op,daily_option_value,option_PL,total_cost_basis,total_pos_value,total_PL,realized_stock_PL,realized_option_PL,realized_PL,UID,to_open
0,2018-01-03,2018-02-02,270.0,270.47,20180202_270.0,C,2.65,2.69,0.072398,0.562218,P,1.77,1.80,0.069568,-0.442522,0.119696,0.119696,0.119696,32.374177,32.374177,32.374177,0.000000,-4.42,-4.42,-4.49,-0.07,27.954177,27.884177,-0.070000,0.000000,0.00,0.000000,20180202_270.0_2018-01-03,1
1,2018-01-04,2018-02-02,270.0,271.61,20180202_270.0,C,3.45,3.49,0.077848,0.632690,P,1.43,1.46,0.074706,-0.366922,0.265768,0.265768,0.146072,39.674616,72.048793,72.185246,0.136453,-4.42,0.00,-4.95,-0.53,67.628793,67.235246,-0.393547,0.000000,0.00,0.000000,20180202_270.0_2018-01-03,0
2,2018-01-05,2018-02-02,270.0,273.42,20180202_270.0,C,4.80,4.84,0.083820,0.727658,P,1.02,1.04,0.081783,-0.270119,0.457539,0.457539,0.191771,52.434027,124.482820,125.100313,0.617494,-4.42,0.00,-5.88,-1.46,120.062820,119.220313,-0.842506,0.000000,0.00,0.000000,20180202_270.0_2018-01-03,0
3,2018-01-08,2018-02-02,270.0,273.92,20180202_270.0,C,5.17,5.20,0.089623,0.749240,P,0.81,0.82,0.082307,-0.234696,0.514544,0.514544,0.057005,15.614810,140.097629,140.943892,0.846263,-4.42,0.00,-6.02,-1.60,135.677629,134.923892,-0.753737,0.000000,0.00,0.000000,20180202_270.0_2018-01-03,0
4,2018-01-09,2018-02-02,270.0,274.54,20180202_270.0,C,5.67,5.72,0.093639,0.773171,P,0.83,0.84,0.091492,-0.223283,0.549888,0.549888,0.035344,9.703342,149.800971,150.966252,1.165280,-4.42,0.00,-6.56,-2.14,145.380971,144.406252,-0.974720,0.000000,0.00,0.000000,20180202_270.0_2018-01-03,0
5,2018-01-10,2018-02-02,270.0,274.12,20180202_270.0,C,5.27,5.33,0.092791,0.759070,P,0.85,0.87,0.089866,-0.235707,0.523363,0.523363,-0.026525,-7.271033,142.529938,143.464266,0.934327,-4.42,0.00,-6.20,-1.78,138.109938,137.264266,-0.845673,0.000000,0.00,0.000000,20180202_270.0_2018-01-03,0
6,2018-01-11,2018-02-02,270.0,276.12,20180202_270.0,C,6.76,6.91,0.090857,0.854846,P,0.56,0.58,0.097319,-0.162272,0.692574,0.692574,0.169211,46.722541,189.252480,191.233533,1.981053,-4.42,0.00,-7.49,-3.07,184.832480,183.743533,-1.088947,0.000000,0.00,0.000000,20180202_270.0_2018-01-03,0
7,2018-01-12,2018-02-02,270.0,277.92,20180202_270.0,C,8.45,8.60,0.102281,0.890014,P,0.43,0.44,0.107499,-0.121856,0.768158,0.768158,0.075584,21.006305,210.258785,213.486471,3.227687,-4.42,0.00,-9.04,-4.62,205.838785,204.446471,-1.392313,0.000000,0.00,0.000000,20180202_270.0_2018-01-03,0
8,2018-01-16,2018-02-02,270.0,276.97,20180202_270.0,C,7.82,7.97,0.130638,0.827166,P,0.60,0.61,0.121851,-0.157003,0.670163,0.670163,-0.097995,-27.141675,183.117110,185.615046,2.497936,-4.42,0.00,-8.58,-4.16,178.697110,177.035046,-1.662064,0.000000,0.00,0.000000,20180202_270.0_2018-01-03,0
9,2018-01-17,2018-02-02,270.0,279.61,20180202_270.0,C,10.03,10.25,0.131682,0.904279,P,0.38,0.39,0.135023,-0.101488,0.802791,0.802791,0.132628,37.084115,220.201225,224.468392,4.267167,-4.42,0.00,-10.64,-6.22,215.781225,213.828392,-1.952833,0.000000,0.00,0.000000,20180202_270.0_2018-01-03,0


## Trading Strat / Positions / Limits

### Trading Strat

In [14]:
iv_data = pd.read_csv('iv_calculations.csv')
iv_data['iv'] = iv_data['iv']**0.5
iv_data

,Unnamed: 0,date,iv
0,0,2018-01-02,0.021847
1,1,2018-01-03,0.023454
2,2,2018-01-04,0.027852
3,3,2018-01-05,0.027788
4,4,2018-01-08,0.029469
...,...,...,...
1293,1293,2023-02-22,0.107065
1294,1294,2023-02-23,0.111236
1295,1295,2023-02-24,0.100687
1296,1296,2023-02-27,0.114114


In [69]:
temp_data = []

for key, df in filtered_simulations.items():
    temp_data.append({'date': key, 'BS_Call_IV': df.loc[0, 'impl_volatility_c']})

BS_Call_IV = pd.DataFrame(temp_data)

iv_data['date'] = pd.to_datetime(iv_data['date'])
BS_Call_IV['date'] = pd.to_datetime(BS_Call_IV['date'])

IV_compare = pd.merge(BS_Call_IV, iv_data[['date', 'iv']], on='date', how='left')
IV_compare.rename(columns={'iv': 'MF_Call_IV'}, inplace=True)
IV_compare['IV_diff'] = IV_compare['MF_Call_IV'] - IV_compare['BS_Call_IV']

In [70]:
IV_compare

,date,BS_Call_IV,MF_Call_IV,IV_diff
0,2018-01-03,0.072398,0.023454,-0.048944
1,2018-01-04,0.071179,0.027852,-0.043327
2,2018-01-10,0.080641,0.030803,-0.049838
3,2018-01-11,0.076890,0.028318,-0.048572
4,2018-01-12,0.079350,0.029221,-0.050129
...,...,...,...,...
968,2023-02-22,0.202075,0.107065,-0.095010
969,2023-02-23,0.196720,0.111236,-0.085484
970,2023-02-24,0.201088,0.100687,-0.100401
971,2023-02-27,0.194675,0.114114,-0.080561


Taking a look at the differences in IV:

In [71]:
(IV_compare['IV_diff'] > 0).sum()

124

In [72]:
IV_compare[IV_compare['IV_diff'] > 0]['IV_diff'].quantile(0.5)

0.3198208484266928

In [73]:
IV_compare[IV_compare['IV_diff'] > 0]['IV_diff'].quantile(0.25)

0.16191266523189105

In [74]:
IV_compare[IV_compare['IV_diff'] > 0]['IV_diff'].quantile(0.2)

0.12497667184676822

In [75]:
IV_compare[IV_compare['IV_diff'] < 0]['IV_diff'].quantile(0.75)

-0.0785731074996433

In [76]:
IV_compare[IV_compare['IV_diff'] < 0]['IV_diff'].quantile(0.80)

-0.07246443270542155

In [77]:
IV_compare[IV_compare['IV_diff'] < 0]['IV_diff'].quantile(0.70)

-0.08366817795015716

In [78]:
IV_compare[IV_compare['IV_diff'] < 0]['IV_diff'].quantile(0.50)

-0.11270668550109814

Trading Strategies:

In [79]:
# Long-Short
def trade_strategy_1(x):
    if x > 0.25:
        return 1
    elif x < -0.08:
        return -1
    else:
        return 0

# Long Only
def trade_strategy_2(x):
    if x > 0.15:
        return 1
    else:
        return 0

# Short Only
def trade_strategy_3(x):
    if x < -0.1:
        return -1
    else:
        return 0

In [80]:
strat_dict = {'trade_1': trade_strategy_1, 'trade_2': trade_strategy_2, 'trade_3': trade_strategy_3}

In [81]:
for key, func in strat_dict.items():
    IV_compare[key] = IV_compare['IV_diff'].apply(func)
IV_compare

,date,BS_Call_IV,MF_Call_IV,IV_diff,trade_1,trade_2,trade_3
0,2018-01-03,0.072398,0.023454,-0.048944,0,0,0
1,2018-01-04,0.071179,0.027852,-0.043327,0,0,0
2,2018-01-10,0.080641,0.030803,-0.049838,0,0,0
3,2018-01-11,0.076890,0.028318,-0.048572,0,0,0
4,2018-01-12,0.079350,0.029221,-0.050129,0,0,0
...,...,...,...,...,...,...,...
968,2023-02-22,0.202075,0.107065,-0.095010,-1,0,0
969,2023-02-23,0.196720,0.111236,-0.085484,-1,0,0
970,2023-02-24,0.201088,0.100687,-0.100401,-1,0,-1
971,2023-02-27,0.194675,0.114114,-0.080561,-1,0,0


In [82]:
IV_compare['date'] = IV_compare['date'].dt.strftime('%Y-%m-%d')
abs(IV_compare['trade_3']).sum()

490

In [83]:
simulations_long[IV_compare['date'].iloc[0]]

,date,exdate,strike_price,close,strikeID,cp_flag_c,best_bid_c,best_offer_c,impl_volatility_c,delta_c,cp_flag_p,best_bid_p,best_offer_p,impl_volatility_p,delta_p,delta_sum,stock_pos,pos_change,change_cost_basis,stock_cost_basis,daily_stock_value,stock_PL,option_cost_basis,change_cost_basis_op,daily_option_value,option_PL,total_cost_basis,total_pos_value,total_PL,realized_stock_PL,realized_option_PL,realized_PL,UID,to_open
0,2018-01-03,2018-02-02,270.0,270.47,20180202_270.0,C,2.65,2.69,0.072398,0.562218,P,1.77,1.80,0.069568,-0.442522,0.119696,-0.119696,-0.119696,-32.374177,-32.374177,-32.374177,0.000000,4.49,4.49,4.42,-0.07,-27.884177,-27.954177,-0.070000,0.000000,0.00,0.000000,20180202_270.0_2018-01-03,1
1,2018-01-04,2018-02-02,270.0,271.61,20180202_270.0,C,3.45,3.49,0.077848,0.632690,P,1.43,1.46,0.074706,-0.366922,0.265768,-0.265768,-0.146072,-39.674616,-72.048793,-72.185246,-0.136453,4.49,0.00,4.88,0.39,-67.558793,-67.305246,0.253547,0.000000,0.00,0.000000,20180202_270.0_2018-01-03,0
2,2018-01-05,2018-02-02,270.0,273.42,20180202_270.0,C,4.80,4.84,0.083820,0.727658,P,1.02,1.04,0.081783,-0.270119,0.457539,-0.457539,-0.191771,-52.434027,-124.482820,-125.100313,-0.617494,4.49,0.00,5.82,1.33,-119.992820,-119.280313,0.712506,0.000000,0.00,0.000000,20180202_270.0_2018-01-03,0
3,2018-01-08,2018-02-02,270.0,273.92,20180202_270.0,C,5.17,5.20,0.089623,0.749240,P,0.81,0.82,0.082307,-0.234696,0.514544,-0.514544,-0.057005,-15.614810,-140.097629,-140.943892,-0.846263,4.49,0.00,5.98,1.49,-135.607629,-134.963892,0.643737,0.000000,0.00,0.000000,20180202_270.0_2018-01-03,0
4,2018-01-09,2018-02-02,270.0,274.54,20180202_270.0,C,5.67,5.72,0.093639,0.773171,P,0.83,0.84,0.091492,-0.223283,0.549888,-0.549888,-0.035344,-9.703342,-149.800971,-150.966252,-1.165280,4.49,0.00,6.50,2.01,-145.310971,-144.466252,0.844720,0.000000,0.00,0.000000,20180202_270.0_2018-01-03,0
5,2018-01-10,2018-02-02,270.0,274.12,20180202_270.0,C,5.27,5.33,0.092791,0.759070,P,0.85,0.87,0.089866,-0.235707,0.523363,-0.523363,0.026525,7.271033,-142.529938,-143.464266,-0.934327,4.49,0.00,6.12,1.63,-138.039938,-137.344266,0.695673,0.000000,0.00,0.000000,20180202_270.0_2018-01-03,0
6,2018-01-11,2018-02-02,270.0,276.12,20180202_270.0,C,6.76,6.91,0.090857,0.854846,P,0.56,0.58,0.097319,-0.162272,0.692574,-0.692574,-0.169211,-46.722541,-189.252480,-191.233533,-1.981053,4.49,0.00,7.32,2.83,-184.762480,-183.913533,0.848947,0.000000,0.00,0.000000,20180202_270.0_2018-01-03,0
7,2018-01-12,2018-02-02,270.0,277.92,20180202_270.0,C,8.45,8.60,0.102281,0.890014,P,0.43,0.44,0.107499,-0.121856,0.768158,-0.768158,-0.075584,-21.006305,-210.258785,-213.486471,-3.227687,4.49,0.00,8.88,4.39,-205.768785,-204.606471,1.162313,0.000000,0.00,0.000000,20180202_270.0_2018-01-03,0
8,2018-01-16,2018-02-02,270.0,276.97,20180202_270.0,C,7.82,7.97,0.130638,0.827166,P,0.60,0.61,0.121851,-0.157003,0.670163,-0.670163,0.097995,27.141675,-183.117110,-185.615046,-2.497936,4.49,0.00,8.42,3.93,-178.627110,-177.195046,1.432064,0.000000,0.00,0.000000,20180202_270.0_2018-01-03,0
9,2018-01-17,2018-02-02,270.0,279.61,20180202_270.0,C,10.03,10.25,0.131682,0.904279,P,0.38,0.39,0.135023,-0.101488,0.802791,-0.802791,-0.132628,-37.084115,-220.201225,-224.468392,-4.267167,4.49,0.00,10.41,5.92,-215.711225,-214.058392,1.652833,0.000000,0.00,0.000000,20180202_270.0_2018-01-03,0


In [84]:
def generate_trades_dfs(strat_dict = strat_dict, initial_df = IV_compare):
    
    trades_dfs = {}
    
    for key in strat_dict.keys():

        dfs_to_combine = []
        
        for index, row in initial_df.iterrows():
            date = row['date']
            trade = row[key]
            iv_diff = row['IV_diff']
            
            if trade == 1 and date in simulations_long:
                df_to_add = simulations_long[date].copy()
            elif trade == -1 and date in simulations_short:
                df_to_add = simulations_short[date].copy()
            else:
                # Skip if 'trade' is 0 or the date is not in the dictionaries
                continue
            
            # Add 'trade' & 'IV_diff' column
            df_to_add['IV_diff'] = iv_diff # Needed for position calculation 
            df_to_add[key] = trade  # Include the 'trade' value
            dfs_to_combine.append(df_to_add)
        
        # Concatenate all collected DataFrames
        trades_dfs[key] = pd.concat(dfs_to_combine, ignore_index=True)
        trades_dfs[key] = trades_dfs[key].sort_values(by=['date', 'exdate', 'strike_price', 'to_open']).reset_index(drop=True)

    return trades_dfs

In [85]:
%%time
trades_dfs = generate_trades_dfs()

CPU times: total: 938 ms
Wall time: 938 ms


In [86]:
KAPITAL = 1e7

In [102]:
def pos_size(IV_diff, strike_price, option_cost_basis, UID, key):
    factor = 10* strike_price # Adjust this based on whatever

    if option_cost_basis == 0:
        # Get the first matching 'option_cost_basis' safely
        filtered_df = trades_dfs[key].loc[trades_dfs[key]['UID'] == UID, 'option_cost_basis']

        option_cost_basis = filtered_df.iloc[0] 

    posSize = round((abs(IV_diff) / factor) * KAPITAL / abs(option_cost_basis+.000000000001)) # some too close to zero, trigger infinity
    
    return posSize
    
    # kept it simple instead of my earlier complex one

In [88]:
for key, df in trades_dfs.items():
    trades_dfs[key] = df.drop(columns=[col for col in df.columns if col.endswith('_p') or col.endswith('_c')]).copy()

In [103]:
for key, df in trades_dfs.items():

    df['pos_size'] = df.apply(lambda row: pos_size(row['IV_diff'], row['strike_price'], row['option_cost_basis'], row['UID'], key), axis=1)
    lot_size = 100 * df['pos_size']
    
    for col in ['stock_pos',	'pos_change',	'change_cost_basis',	'stock_cost_basis',	'daily_stock_value',	'stock_PL',	'option_cost_basis',
                'change_cost_basis_op', 'daily_option_value',	'option_PL',	'total_cost_basis',	'total_pos_value',	'total_PL',	'realized_stock_PL',
                'realized_option_PL',	'realized_PL']:
        df['sized_' + col] = lot_size * df[col]

In [110]:
trades_dfs['trade_1'].head(100)

,date,exdate,strike_price,close,strikeID,delta_sum,stock_pos,pos_change,change_cost_basis,stock_cost_basis,daily_stock_value,stock_PL,option_cost_basis,change_cost_basis_op,daily_option_value,option_PL,total_cost_basis,total_pos_value,total_PL,realized_stock_PL,realized_option_PL,realized_PL,UID,to_open,IV_diff,trade_1,pos_size,sized_stock_pos,sized_pos_change,sized_change_cost_basis,sized_stock_cost_basis,sized_daily_stock_value,sized_stock_PL,sized_option_cost_basis,sized_change_cost_basis_op,sized_daily_option_value,sized_option_PL,sized_total_cost_basis,sized_total_pos_value,sized_total_PL,sized_realized_stock_PL,sized_realized_option_PL,sized_realized_PL,signed_pos,signed_pos_stock
0,2018-02-05,2018-03-07,265.0,263.93,20180307_265.0,0.001757,0.001757,0.001757,0.463725,0.463725,0.463725,0.000000,-16.23,-16.23,-17.00,-0.77,-15.766275,-16.536275,-0.770000,0.0,0.0,0.0,20180307_265.0_2018-02-05,1,-0.294289,-1,68,11.9476,11.9476,3.153330e+03,3.153330e+03,3.153330e+03,0.000000,-110364.0,-110364.0,-115600.0,-5236.0,-1.072107e+05,-1.124467e+05,-5236.000000,0.0,0.0,0.0,68,11.9476
1,2018-02-06,2018-03-07,265.0,269.13,20180307_265.0,0.224091,0.224091,0.222334,59.836749,60.300474,60.309611,0.009136,-16.23,0.00,-14.83,1.40,44.070474,45.479611,1.409136,0.0,0.0,0.0,20180307_265.0_2018-02-05,0,-0.294289,-1,68,1523.8188,1511.8712,4.068899e+05,4.100432e+05,4.101054e+05,62.127520,-110364.0,0.0,-100844.0,9520.0,2.996792e+05,3.092614e+05,9582.127520,0.0,0.0,0.0,68,1523.8188
2,2018-02-06,2018-03-07,269.0,269.13,20180307_269.0,0.045835,0.045835,0.045835,12.335574,12.335574,12.335574,0.000000,-13.27,-13.27,-13.75,-0.48,-0.934426,-1.414426,-0.480000,0.0,0.0,0.0,20180307_269.0_2018-02-06,1,-0.191665,-1,54,247.5090,247.5090,6.661210e+04,6.661210e+04,6.661210e+04,0.000000,-71658.0,-71658.0,-74250.0,-2592.0,-5.045903e+03,-7.637903e+03,-2592.000000,0.0,0.0,0.0,54,247.5090
3,2018-02-07,2018-03-07,265.0,267.67,20180307_265.0,0.189307,0.189307,-0.034784,-9.310633,50.989841,50.671805,-0.318036,-16.23,0.00,-11.60,4.63,34.759841,39.071805,4.311964,0.0,0.0,0.0,20180307_265.0_2018-02-05,0,-0.294289,-1,68,1287.2876,-236.5312,-6.331231e+04,3.467309e+05,3.445683e+05,-2162.647928,-110364.0,0.0,-78880.0,31484.0,2.363669e+05,2.656883e+05,29321.352072,0.0,0.0,0.0,68,1287.2876
4,2018-02-07,2018-03-07,269.0,267.67,20180307_269.0,-0.048254,-0.048254,-0.094089,-25.184803,-12.849229,-12.916148,-0.066919,-13.27,0.00,-10.23,3.04,-26.119229,-23.146148,2.973081,0.0,0.0,0.0,20180307_269.0_2018-02-06,0,-0.191665,-1,54,-260.5716,-508.0806,-1.359979e+05,-6.938584e+04,-6.974720e+04,-361.363140,-71658.0,0.0,-55242.0,16416.0,-1.410438e+05,-1.249892e+05,16054.636860,0.0,0.0,0.0,54,-260.5716
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2018-02-27,2018-03-09,261.0,274.43,20180309_261.0,0.777347,0.777347,-0.161432,-44.301784,209.623220,213.327337,3.704117,-13.12,0.00,-14.99,-1.87,196.503220,198.337337,1.834117,0.0,0.0,0.0,20180309_261.0_2018-02-09,0,-0.187725,-1,55,4275.4085,-887.8760,-2.436598e+05,1.152928e+06,1.173300e+06,20372.645205,-72160.0,0.0,-82445.0,-10285.0,1.080768e+06,1.090855e+06,10087.645205,0.0,0.0,0.0,55,4275.4085
96,2018-02-27,2018-03-09,267.5,274.43,20180309_267.5,0.568180,0.568180,-0.243973,-66.953510,159.527269,155.925637,-3.601632,-10.59,0.00,-9.51,1.08,148.937269,146.415637,-2.521632,0.0,0.0,0.0,20180309_267.5_2018-02-07,0,-0.163611,-1,58,3295.4440,-1415.0434,-3.883304e+05,9.252582e+05,9.043687e+05,-20889.464846,-61422.0,0.0,-55158.0,6264.0,8.638362e+05,8.492107e+05,-14625.464846,0.0,0.0,0.0,58,3295.4440
97,2018-02-27,2018-03-14,265.0,274.43,20180314_265.0,0.618176,0.618176,-0.207205,-56.863268,168.351944,169.646040,1.294095,-10.95,0.00,-12.18,-1.23,157.401944,157.466040,0.064095,0.0,0.0,0.0,20180314_265.0_2018-02-12,0,-0.113292,-1,39,2410.8864,-808.0995,-2.217667e+05,6.565726e+05,6.616196e+05,5046.971397,-42705.0,0.0,-47502.

### PL Positions

In [122]:
PL_temp_dfs = {}  
for key, df in trades_dfs.items():
    # Group by 'dates' and sum the columns of interest
    columns_to_sum = [] 
    for col in ['stock_pos', 'change_cost_basis',	'stock_cost_basis',	'daily_stock_value',	'stock_PL',	'option_cost_basis', 'change_cost_basis_op',
                'daily_option_value',	'option_PL',	'total_cost_basis',	'total_pos_value',	'total_PL',	'realized_stock_PL',
                'realized_option_PL',	'realized_PL']:
        columns_to_sum.append('sized_' + col)
    
    
    grouped_df = df[['date'] + columns_to_sum].groupby('date').sum().reset_index()
    
    # Reindex the resulting DataFrame to match 'trading_days', filling missing values with 0 or other appropriate value
    pl_df = grouped_df.set_index('date').reindex(trading_days).fillna(0).reset_index()
    pl_df.rename(columns={'index': 'date'}, inplace=True)
    
    # Assign the processed DataFrame to PL_dfs with the same key
    PL_temp_dfs[key] = pl_df
    
PL_temp_dfs['trade_1'][50:84]

,date,sized_stock_pos,sized_change_cost_basis,sized_stock_cost_basis,sized_daily_stock_value,sized_stock_PL,sized_option_cost_basis,sized_change_cost_basis_op,sized_daily_option_value,sized_option_PL,sized_total_cost_basis,sized_total_pos_value,sized_total_PL,sized_realized_stock_PL,sized_realized_option_PL,sized_realized_PL
50,2018-03-15,-13453.7423,3.930371e+05,-3.778990e+06,-3.699779e+06,79210.610443,426893.0,30086.0,284831.0,-142062.0,-3.352097e+06,-3.414948e+06,-62851.389557,-15126.630554,13965.0,-1161.630554
51,2018-03-16,-13877.4692,-1.161859e+05,-3.895176e+06,-3.805202e+06,89973.604283,426893.0,0.0,273151.0,-153742.0,-3.468283e+06,-3.532051e+06,-63768.395717,0.000000,0.0,0.000000
52,2018-03-19,5829.7840,5.330615e+06,1.435439e+06,1.576898e+06,141459.015015,426893.0,0.0,268598.0,-158295.0,1.862332e+06,1.845496e+06,-16835.984985,0.000000,0.0,0.000000
53,2018-03-20,-2750.0376,-2.575206e+06,-8.336709e+05,-7.451227e+05,88548.258884,279052.0,-82641.0,167884.0,-111168.0,-5.546189e+05,-5.772387e+05,-22619.741116,55592.456771,-65200.0,-9607.543229
54,2018-03-21,-4088.0844,3.203582e+05,-1.139275e+06,-1.105541e+06,33734.724036,134412.0,-73120.0,80454.0,-53958.0,-1.004863e+06,-1.025087e+06,-20223.275964,56243.554400,-71520.0,-15276.445600
55,2018-03-22,77.0112,-1.037505e+05,2.030554e+04,2.030554e+04,0.000000,-38916.0,-128064.0,-39492.0,-576.0,-1.861046e+04,-1.918646e+04,-576.000000,61370.174580,-45264.0,16106.174580
56,2018-03-23,-1403.1080,-3.819448e+05,-3.616392e+05,-3.620720e+05,-432.802944,-85276.0,-46360.0,-89000.0,-3724.0,-4.469152e+05,-4.510720e+05,-4156.802944,0.000000,0.0,0.000000
57,2018-03-26,1969.0906,8.940036e+05,5.323644e+05,5.220256e+05,-10338.745424,-139283.0,-54007.0,-138578.0,705.0,3.930814e+05,3.834476e+05,-9633.745424,0.000000,0.0,0.000000
58,2018-03-27,-1155.8181,-8.143512e+05,-2.819869e+05,-3.012062e+05,-19219.344030,-201869.0,-62586.0,-202287.0,-418.0,-4.838559e+05,-5.034932e+05,-19637.344030,0.000000,0.0,0.000000
59,2018-03-28,-1876.4168,-1.872332e+05,-4.692200e+05,-4.875494e+05,-18329.364093,-257484.0,-55615.0,-259296.0,-1812.0,-7.267040e+05,-7.468454e+05,-20141.364093,0.000000,0.0,0.000000


In [123]:
divvies = {}

for key, df in PL_temp_dfs.items():
    # Ensure 'date' in both DataFrames is in datetime format
    df['date'] = pd.to_datetime(df['date'])
    spy_divdata['date'] = pd.to_datetime(spy_divdata['date'])
    spy_divdata['pay_date'] = pd.to_datetime(spy_divdata['pay_date'])
    
    # First Merge: Merge to align 'pos_size' with 'spy_divdata' based on 'date'
    # Assuming 'pos_size' exists in your df
    temp_merged = pd.merge(spy_divdata, df[['date', 'sized_stock_pos']], how='left', on='date')

    # Correctly multiply 'signed_pos' by 'dividend' column
    temp_merged['div'] = temp_merged['sized_stock_pos'] * temp_merged['dividend']
    
    # Update the DataFrame in divvies with the final merged DataFrame
    divvies[key] = temp_merged

divvies['trade_1'].head(50)

,date,dividend,pay_date,sized_stock_pos,div
0,2018-03-16,1.096780,2018-04-30,-13877.4692,-15220.530669
1,2018-06-15,1.246000,2018-07-31,0.0000,0.000000
2,2018-09-21,1.323000,2018-10-31,-3845.2722,-5087.295121
3,2018-12-21,1.435400,2019-01-31,-7903.2848,-11344.375002
4,2019-03-15,1.233100,2019-04-30,-9631.3812,-11876.456158
5,2019-06-21,1.431600,2019-07-31,1566.5728,2242.705620
6,2019-09-20,1.383620,2019-10-31,0.0000,0.000000
7,2019-12-20,1.570000,2020-01-31,-3220.6182,-5056.370574
8,2020-03-20,1.405560,2020-04-30,-6867.6849,-9652.943188
9,2020-06-19,1.366200,2020-07-31,1968.4825,2689.340791


In [124]:
for key, pl_df in PL_temp_dfs.items():
    
    # Get the corresponding DataFrame from divvies
    div_df = divvies[key]
    pl_df['date'] = pd.to_datetime(pl_df['date'])
    # Perform the left merge
    # Note: Ensure 'date' in PL_temp_dfs and 'pay_date' in divvies are of compatible types (preferably datetime)
    merged_df = pd.merge(pl_df, div_df[['pay_date', 'div']], how='left', left_on='date', right_on='pay_date')
    
    # Optionally, you can drop the 'pay_date' column after the merge if it's no longer needed
    merged_df.drop(columns=['pay_date'], inplace=True)
    
    # Assuming 'div' can be NaN, fill NaNs with 0 for accurate calculations
    merged_df['div'] = merged_df['div'].fillna(0)
    
    # Update 'sized_realized_stock_PL' and 'sized_realized_PL' by adding 'div'
    merged_df['sized_realized_stock_PL'] = merged_df['sized_realized_stock_PL'] + merged_df['div']
    merged_df['sized_realized_PL'] = merged_df['sized_realized_PL'] + merged_df['div']
    
    # Update the PL_temp_dfs dictionary with the merged and updated result
    PL_temp_dfs[key] = merged_df

In [125]:
PL_temp_dfs['trade_1'][50:84]

,date,sized_stock_pos,sized_change_cost_basis,sized_stock_cost_basis,sized_daily_stock_value,sized_stock_PL,sized_option_cost_basis,sized_change_cost_basis_op,sized_daily_option_value,sized_option_PL,sized_total_cost_basis,sized_total_pos_value,sized_total_PL,sized_realized_stock_PL,sized_realized_option_PL,sized_realized_PL,div
50,2018-03-15,-13453.7423,3.930371e+05,-3.778990e+06,-3.699779e+06,79210.610443,426893.0,30086.0,284831.0,-142062.0,-3.352097e+06,-3.414948e+06,-62851.389557,-15126.630554,13965.0,-1161.630554,0.000000
51,2018-03-16,-13877.4692,-1.161859e+05,-3.895176e+06,-3.805202e+06,89973.604283,426893.0,0.0,273151.0,-153742.0,-3.468283e+06,-3.532051e+06,-63768.395717,0.000000,0.0,0.000000,0.000000
52,2018-03-19,5829.7840,5.330615e+06,1.435439e+06,1.576898e+06,141459.015015,426893.0,0.0,268598.0,-158295.0,1.862332e+06,1.845496e+06,-16835.984985,0.000000,0.0,0.000000,0.000000
53,2018-03-20,-2750.0376,-2.575206e+06,-8.336709e+05,-7.451227e+05,88548.258884,279052.0,-82641.0,167884.0,-111168.0,-5.546189e+05,-5.772387e+05,-22619.741116,55592.456771,-65200.0,-9607.543229,0.000000
54,2018-03-21,-4088.0844,3.203582e+05,-1.139275e+06,-1.105541e+06,33734.724036,134412.0,-73120.0,80454.0,-53958.0,-1.004863e+06,-1.025087e+06,-20223.275964,56243.554400,-71520.0,-15276.445600,0.000000
55,2018-03-22,77.0112,-1.037505e+05,2.030554e+04,2.030554e+04,0.000000,-38916.0,-128064.0,-39492.0,-576.0,-1.861046e+04,-1.918646e+04,-576.000000,61370.174580,-45264.0,16106.174580,0.000000
56,2018-03-23,-1403.1080,-3.819448e+05,-3.616392e+05,-3.620720e+05,-432.802944,-85276.0,-46360.0,-89000.0,-3724.0,-4.469152e+05,-4.510720e+05,-4156.802944,0.000000,0.0,0.000000,0.000000
57,2018-03-26,1969.0906,8.940036e+05,5.323644e+05,5.220256e+05,-10338.745424,-139283.0,-54007.0,-138578.0,705.0,3.930814e+05,3.834476e+05,-9633.745424,0.000000,0.0,0.000000,0.000000
58,2018-03-27,-1155.8181,-8.143512e+05,-2.819869e+05,-3.012062e+05,-19219.344030,-201869.0,-62586.0,-202287.0,-418.0,-4.838559e+05,-5.034932e+05,-19637.344030,0.000000,0.0,0.000000,0.000000
59,2018-03-28,-1876.4168,-1.872332e+05,-4.692200e+05,-4.875494e+05,-18329.364093,-257484.0,-55615.0,-259296.0,-1812.0,-7.267040e+05,-7.468454e+05,-20141.364093,0.000000,0.0,0.000000,0.000000


In [126]:
PL_dfs = {}  # New dictionary to store the processed DataFrames

for key, df in PL_temp_dfs.items():
    # Create a new DataFrame for PL_dfs
    pl_df = pd.DataFrame(index=df.index)
    
    # Assuming 'date' is the column you want to keep as is
    pl_df['date'] = df['date']
    
    # Sum 'sized_change_cost_basis' and 'sized_change_cost_basis_op'
    pl_df['gross_trades_value'] = abs(df['sized_change_cost_basis']) + abs(df['sized_change_cost_basis_op'])
    
    # Calculate cumulative sum of 'sized_realized_stock_PL'
    pl_df['stock_PL'] = df['sized_realized_stock_PL'].cumsum()
    pl_df['option_PL'] = df['sized_realized_option_PL'].cumsum()
    pl_df['net_PL'] = df['sized_realized_PL'].cumsum()

    # Positions & Cost Basis
    pl_df['cash'] = KAPITAL - df['sized_total_cost_basis'] + pl_df['net_PL']
    pl_df['position_value'] = pl_df['cash'] + df['sized_total_pos_value']
    
    # Assign the processed DataFrame to the new dictionary
    PL_dfs[key] = pl_df

In [127]:
PL_dfs['trade_1'][50:85]

,date,gross_trades_value,stock_PL,option_PL,net_PL,cash,position_value
50,2018-03-15,4.231231e+05,47787.623063,-21400.0,26387.623063,1.337848e+07,9.963536e+06
51,2018-03-16,1.161859e+05,47787.623063,-21400.0,26387.623063,1.349467e+07,9.962619e+06
52,2018-03-19,5.330615e+06,47787.623063,-21400.0,26387.623063,8.164055e+06,1.000955e+07
53,2018-03-20,2.657847e+06,103380.079834,-86600.0,16780.079834,1.057140e+07,9.994160e+06
54,2018-03-21,3.934782e+05,159623.634234,-158120.0,1503.634234,1.100637e+07,9.981280e+06
55,2018-03-22,2.318145e+05,220993.808814,-203384.0,17609.808814,1.003622e+07,1.001703e+07
56,2018-03-23,4.283048e+05,220993.808814,-203384.0,17609.808814,1.046453e+07,1.001345e+07
57,2018-03-26,9.480106e+05,220993.808814,-203384.0,17609.808814,9.624528e+06,1.000798e+07
58,2018-03-27,8.769372e+05,220993.808814,-203384.0,17609.808814,1.050147e+07,9.997972e+06
59,2018-03-28,2.428482e+05,220993.808814,-203384.0,17609.808814,1.074431e+07,9.997468e+06


In [140]:
PL_dfs['trade_1'][-2:-1]

,date,gross_trades_value,stock_PL,option_PL,net_PL,cash,position_value
1296,2023-02-27,239239.723911,-923678.074152,1760256.0,836577.925848,1.099554e+07,1.084416e+07


In [129]:
os.makedirs('simdata', exist_ok=True)

for strat, df in PL_dfs.items():
    csv_path = f'simdata/PL_{strat}.csv'
    df.to_csv(csv_path, index=False)

### Other things to consider (WIP)

In [ ]:
# Aggregate same strike/expiration options to further reduce trading costs once positions have been determined, etc.
for key, df in trades_dfs.items():
    df = df.drop(columns=[col for col in df.columns if col.endswith('_p') or col.endswith('_c')])

aggregations = {
    'exdate': lambda x: x.iloc[0] if all(date == x.iloc[0] for date in x) else 'Variable',
    'strike_price': lambda x: x.iloc[0] if all(strike == x.iloc[0] for strike in x) else None,
    'close': lambda x: x.iloc[0] if all(strike == x.iloc[0] for strike in x) else None,
    'delta_sum':
    stock_pos
    pos_change
    change_cost_basis
    stock_cost_basis
    daily_stock_value
    stock_PL
    option_cost_basis
    daily_option_value
    option_PL
    total_cost_basis
    total_pos_value	total_PL
    realized_stock_PL
    realized_option_PL
    realized_PL
    UID
    to_open
    IV_diff	
}